In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("../..")
# from parc.data import EnergeticMatDataPipeLine as EmData
from parc import misc, metrics, model,visualization
from parc.model import model_burgers
from skimage.measure import block_reduce



2024-01-07 11:40:39.913139: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data pipeline

In [2]:
import time
import os
import numpy as np
import skimage
from skimage.measure import block_reduce


Re_list = [15,20,30,40,60,80,100,120,140,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]#,2000,3000,4000,5000,6000,7000,8000,9000,10000]

def clip_raw_data():
    data_whole = []
    r_whole = []
    for Re in Re_list:
        data_file_name = 'Re_' + str(int(Re)) + '.npy'
        file_path = './ns_data/' + data_file_name                
        if os.path.exists(file_path):
            raw_data = np.float32(np.load(file_path))
            raw_data = np.expand_dims(raw_data, axis = 0)
            data_shape = raw_data.shape
            norm_r = Re/1000
            r_img = norm_r*np.ones(shape = (data_shape[0],data_shape[1],data_shape[2],1))
            r_whole.extend(r_img)
            data_whole.extend(raw_data)

    data_whole = np.concatenate([data_whole], axis=0)
    r_whole = np.concatenate([r_whole], axis=0)
    return data_whole, r_whole

seq_clipped = clip_raw_data()



In [3]:
# Normalization
def data_normalization(input_data,no_of_channel):
    norm_data = np.zeros(input_data.shape)
    min_val = []
    max_val = []
    for i in range(no_of_channel):
        iter_max_val = np.quantile(input_data[:,:,:,i::3],0.995)
        iter_min_val = np.quantile(input_data[:,:,:,i::3],0.005)
        norm_data[:,:,:,(i)::no_of_channel] = ((input_data[:,:,:,(i)::no_of_channel] - iter_min_val)) / (iter_max_val - iter_min_val + 1E-9)
        min_val.append(iter_min_val)
        max_val.append(iter_max_val)
    return norm_data, min_val, max_val

In [4]:
seq_clipped[0].shape
seq_norm = data_normalization(seq_clipped[0], 3)

In [5]:
print(seq_norm[1], seq_norm[2])


[-0.653564119040966, -1.1162888139486313, -4362.601213378905] [1.8618228328227993, 1.1177965223789208, 3100.333607177734]


In [7]:
Re_list = [15,20,30,40,60,80,100,120,140,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
train_list = [30,40,80,100,150,200,250,300,400,450,500,600,650,700,800,850,900,950]
test_list = [20,60,140,350,550,750,1000]
idx = 0
train_idx =[]
test_idx =[]
for Re in Re_list:
    if Re in train_list:
        train_idx.append(idx)
    elif Re in test_list:
        test_idx.append(idx)
    idx += 1

In [8]:
print(test_idx)

[1, 4, 8, 13, 17, 21, 26]


In [9]:
train_seq = [seq_norm[0][idx:idx+1,:,:,:] for idx in train_idx]
train_re =[seq_clipped[1][idx:idx+1,:,:,:] for idx in train_idx]

test_seq = [seq_norm[0][idx:idx+1,:,:,:] for idx in test_idx]
test_re =[seq_clipped[1][idx:idx+1,:,:,:] for idx in test_idx]


In [10]:
train_seq = np.concatenate(train_seq, axis = 0)
train_re = np.concatenate(train_re, axis = 0)

test_seq = np.concatenate(test_seq, axis = 0)
test_re = np.concatenate(test_re, axis = 0)
print(test_seq.shape)

(7, 128, 256, 117)


In [11]:
print(train_re.shape)

(18, 128, 256, 1)


In [10]:
def create_train_data(seq, re, no_of_fields, sequence_length = 2):
    shape = seq.shape
    num_time_steps = np.int32((shape[-1]-1)/3)
    vel_seq_whole = []
    re_seq_whole = []
    for i in range(shape[0]):
        for j in range(num_time_steps-sequence_length+1):
            vel_seq_case = np.expand_dims(seq[i, :, :, (j*no_of_fields):(j*no_of_fields+sequence_length*no_of_fields)],axis = 0)
            vel_seq_whole.extend(vel_seq_case)
            re_seq_whole.extend(re[i:i+1,:,:,:])
    vel_seq_whole = np.concatenate([vel_seq_whole], axis=0)
    re_seq_whole = np.concatenate([re_seq_whole], axis=0)

    return vel_seq_whole,re_seq_whole

train_data,train_re_seq = create_train_data(train_seq, train_re, no_of_fields = 3, sequence_length = 11)

In [12]:
train_re_seq.shape

(594, 128, 256, 1)

In [12]:
from tensorflow import keras
from tensorflow.keras import  layers, regularizers
from keras.layers import *
import tensorflow as tf
from parc import layer

from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.models import Model

def differentiator_neural_ode_ns():
    # Model initiation
    feature_extraction = layer.feature_extraction_unet(input_shape = (128,256), n_out_features = 64, n_base_features = 64, n_channel = 4)
    
    # Main computation graph
    velocity_field = Input(shape=(128,256, 4), dtype = tf.float32)

    # Reaction term
    dynamic_feature = feature_extraction(velocity_field)
    
    # Final mapping
    velocity_dot = Conv2D(2,1, padding = 'same')(dynamic_feature)
    
    differentiator = Model(velocity_field, velocity_dot)
    return differentiator

def poisson_block(input_shape = (128,256),n_base_features = 64):
    inputs = keras.Input(shape = (input_shape[0], input_shape[1], 4), dtype = tf.float32)
    conv = layer.conv_block_down(inputs,
                           feat_dim = n_base_features,
                            reps = 1,
                            kernel_size = 3,
                            mode = 'normal')
    conv2 = layer.conv_block_down(conv,
                           feat_dim = n_base_features*2,
                            reps = 1,
                            kernel_size = 3,
                            mode = 'normal')
    # conv_res = layer.resnet_block(conv, n_base_features, kernel_size = 3, reps = 2, pooling = False)
    conv_out = Conv2D(1,1, padding='same')(conv2)
    poisson = keras.Model([inputs], conv_out)
    return poisson
 
# def integrator_burgers():
#     velocity_integrator = layer.integrator_cnn(input_shape = (128,256), n_base_features = 64, n_output=2)

#     velocity_prev = keras.layers.Input(shape = (128,256, 2), dtype = tf.float32)
#     velocity_dot = keras.layers.Input(shape = (128,256, 2), dtype = tf.float32)

#     velocity_next = velocity_integrator([velocity_dot, velocity_prev])
#     integrator = keras.Model([velocity_dot, velocity_prev], [velocity_next])
#     return integrator

class PARCv2_ns(keras.Model):
    def __init__(self, n_time_step, step_size, solver = "rk4", **kwargs):
        super(PARCv2_ns, self).__init__(**kwargs)
        self.n_time_step = n_time_step
        self.step_size = step_size
        self.solver = solver
        self.differentiator = differentiator_neural_ode_ns()
        self.poisson = poisson_block()

    @property
    def metrics(self):
        return [
        self.total_loss_tracker,
        ]
    
    def call(self, input_data):
        input_seq = tf.cast(input_data[0],dtype = tf.float32)
        reynold_num = tf.cast(input_data[1], dtype = tf.float32)

        res = []
        res.append(input_seq)
        input_seq_current = Concatenate(axis = -1)([input_seq,reynold_num])
        for _ in range(self.n_time_step):    
            pressure = self.poisson(input_seq_current)
            input_seq_current = Concatenate(axis = -1)([input_seq_current[:,:,:,:2], pressure,input_seq_current[:,:,:,3:]])
            input_seq_current, update = self.explicit_update(input_seq_current)
            res.append(input_seq_current[:,:,:,:3])
        output = Concatenate(axis = -1)(res)        
        return output

    @tf.function
    def train_step(self, data):
        velocity_init = tf.cast(data[0][0], dtype = tf.float32)
        reynold_num = tf.cast(data[0][1], dtype = tf.float32)
        velocity_gt = tf.cast(data[1], dtype = tf.float32)

        input_seq_current = Concatenate(axis = -1)([velocity_init,reynold_num])
        with tf.GradientTape() as tape:
            output_snap = []
            for _ in range(self.n_time_step):
                pressure = self.poisson(input_seq_current)
                input_seq_current = Concatenate(axis = -1)([input_seq_current[:,:,:,:2], pressure,input_seq_current[:,:,:,3:]])
                input_seq_current = self.explicit_update(input_seq_current)
                output_snap.append(input_seq_current[:,:,:,:3])
            output = Concatenate(axis = -1)(output_snap)
            total_loss = tf.keras.losses.MeanAbsoluteError(reduction = 'sum')(output,velocity_gt) 
                           
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        return {
            "total_loss": self.total_loss_tracker.result(),
        }
    
    # Update scheme
    def explicit_update(self, input_seq_current):
        input_seq_current = tf.clip_by_value(input_seq_current, 0, 1)
        input_seq_current = self.rk4_update(input_seq_current)
        # if self.solver == "rk4":
        #     input_seq_current, update = self.rk4_update(input_seq_current)
        # elif self.solver == 'heun':
        #     input_seq_current, update = self.heun_update(input_seq_current)
        # else:
        #     input_seq_current, update = self.euler_update(input_seq_current)

        return input_seq_current

    def heun_update(self, input_seq_current):
        # Compute update
        k1 = self.differentiator(input_seq_current)

        # Compute k2       
        inp_k2 = input_seq_current[:,:,:,0:2] + self.step_size*k1
        inp_k2 = Concatenate(axis = -1)([inp_k2,input_seq_current[:,:,:,2:]])

        k2 = self.differentiator(inp_k2)
        
        update = 1/2*(k1 + k2)
        
        final_state = input_seq_current[:,:,:,0:2] + self.step_size*update 
        input_seq_current = Concatenate(axis = -1)([final_state,input_seq_current[:,:,:,2:]])
        return input_seq_current
    

# Training


### Stage 1: Differentiator training

In [12]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices((train_data[:,:,:,:3],train_re_seq))
dataset_label = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 798) 
dataset = dataset.batch(6)

2024-01-03 16:02:01.298195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37939 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [ ]:
tf.keras.backend.clear_session()
parc = PARCv2_ns(n_time_step = 10, step_size= 1/38, solver = "heun")
parc.differentiator.load_weights('parc_diff_ns_heun_5.h5')
parc.poisson.load_weights('parc_poisson_ns_heun_5.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 50, shuffle = True)

In [14]:
parc.differentiator.save_weights('parc_diff_ns_heun_6.h5')
parc.poisson.save_weights('parc_poisson_ns_heun_6.h5')

# Validation

In [17]:
tf.keras.backend.clear_session()
parc = PARCv2_ns(n_time_step = 37, step_size= 1/38, solver = "heun", mode = "differentiator_training")
parc.differentiator.load_weights('parc_diff_ns_heun_6.h5')
parc.poisson.load_weights('parc_poisson_ns_heun_6.h5')
parc.compile()

In [18]:
test_seq.shape

(7, 128, 256, 117)

In [19]:
pred_whole =[]
for idx in range(7):
    input_seq_current = tf.cast(test_seq[idx:idx+1,:,:,:3], dtype = tf.float32)
    output = parc.predict([input_seq_current,test_re[idx:idx+1,:,:,:]])
    pred_whole.append(output)
pred = np.concatenate(pred_whole,axis = 0)

1/1 [==============================] - 0s 122ms/step


In [26]:
pred.shape

(7, 128, 256, 114)

In [20]:
def DeNormalization(y_pred, min_val, max_val, no_of_channel):
    denorm_data = np.zeros(y_pred.shape)
    
    for i in range(no_of_channel):
        print(max_val[i], min_val[i])
        denorm_data[:,:,:,(i)::no_of_channel] = (y_pred[:,:,:,(i)::no_of_channel] * (max_val[i] - min_val[i])) + min_val[i]
    return denorm_data

y_pred_denorm = DeNormalization(pred,seq_norm[1], seq_norm[2], no_of_channel = 3)
# gt_denorm = DeNormalization(test_seq,seq_norm[1], seq_norm[2], no_of_channel = 3)
# print(np.amax(np.sqrt(gt_denorm[3,:,:,0::3]**2 + gt_denorm[3,:,:,1::3]**2)))

1.8618228328227993 -0.653564119040966
1.1177965223789208 -1.1162888139486313
3100.333607177734 -4362.601213378905


In [21]:
np.save('./plotting/ns/parc_ns.npy',y_pred_denorm)